# Exploratory Data Analysis

## Imports and constants

In [1]:
%load_ext autoreload
%autoreload 2

import os
from typing import Dict, Any
import logging 

logging.basicConfig(
     level=logging.INFO, 
     format= '[%(asctime)s|%(levelname)s|%(module)s.py:%(lineno)s] %(message)s',
     datefmt='%H:%M:%S'
 )
import pandas as pd
from tqdm import tqdm
# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()
from transformers import AutoTokenizer
from datasets import load_dataset

from defi_textmine_2025.data import (
    load_labeled_raw_data,
    load_test_raw_data,
    clean_raw_dataset,
    print_value_types,
    save_data,
    TextToMultiLabelDataGenerator,
)
from defi_textmine_2025.data import TARGET_COL, INPUT_COLS, INTERIM_DIR, EDA_DIR

VALIDATION_RATE = 0.25
checkpoint = "camembert/camembert-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def count_tokens(text: str) -> int:
    return len(tokenizer(text)["input_ids"])

/home/tagny/.pyenv/versions/3.11.8/envs/kgl11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[22:45:30|INFO|config.py:58] PyTorch version 2.3.1 available.
[22:45:30|INFO|config.py:105] TensorFlow version 2.16.2 available.
/home/tagny/.pyenv/versions/3.11.8/envs/kgl11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Loading data

In [2]:
labeled_raw_df = load_labeled_raw_data()
labeled_raw_df

,text,entities,relations
id,,,
181,"Anam Destresse, président de l'ONG ""Ma passion...","[{""id"": 0, ""mentions"": [{""value"": ""accident"", ...","[[0, ""STARTED_IN"", 9], [7, ""IS_LOCATED_IN"", 9]..."
31669,"À Paris, le 8 avril 2022, l'usine de déodorant...","[{""id"": 0, ""mentions"": [{""value"": ""explosé"", ""...","[[9, ""IS_LOCATED_IN"", 8], [11, ""OPERATES_IN"", ..."
51470,"En Espagne, dans une région agricole, une cont...","[{""id"": 0, ""mentions"": [{""value"": ""contaminati...","[[7, ""IS_PART_OF"", 8], [9, ""OPERATES_IN"", 1], ..."
51332,Un important incendie a fait des ravages dans ...,"[{""id"": 0, ""mentions"": [{""value"": ""incendie"", ...","[[12, ""IS_IN_CONTACT_WITH"", 5], [0, ""IS_LOCATE..."
1131,« Je coule » : onze heures après avoir envoyé ...,"[{""id"": 0, ""mentions"": [{""value"": ""renversé"", ...","[[9, ""IS_LOCATED_IN"", 2], [0, ""START_DATE"", 17..."
...,...,...,...
1288,"M. Thibault Alekseï, membre de l'association ""...","[{""id"": 0, ""mentions"": [{""value"": ""perdu le co...","[[2, ""IS_PART_OF"", 3], [12, ""IS_IN_CONTACT_WIT..."
3822,Des employés d'une usine de production médical...,"[{""id"": 0, ""mentions"": [{""value"": ""grève"", ""st...","[[8, ""HAS_CONTROL_OVER"", 7], [4, ""HAS_CATEGORY..."
51266,"Le 13 mars 2001, Belgrade a connu un krach bou...","[{""id"": 0, ""mentions"": [{""value"": ""krach bours...","[[1, ""STARTED_IN"", 3], [10, ""IS_PART_OF"", 4], ..."


In [3]:
test_raw_df = load_test_raw_data()
test_raw_df

,text,entities
id,,
1204,“FEAR” est une organisation spécialisée dans l...,"[{""id"": 0, ""mentions"": [{""value"": ""crash"", ""st..."
4909,"48 heures après le braquage du ""DC Supermarket...","[{""id"": 0, ""mentions"": [{""value"": ""DC Supermar..."
2353,"À Genève, la boîte de nuit ""Pretty Woman"" a en...","[{""id"": 0, ""mentions"": [{""value"": ""tombé"", ""st..."
1210,"À Caracas, l'an dernier, une mine située à l'o...","[{""id"": 0, ""mentions"": [{""value"": ""effondremen..."
41948,"À Genève, une industrie de fabrication d'arbre...","[{""id"": 0, ""mentions"": [{""value"": ""incendiée"",..."
...,...,...
51452,"Le WOOP, une organisation mondiale, avait prop...","[{""id"": 0, ""mentions"": [{""value"": ""épidémie"", ..."
51491,"Madame Pautrat Elina, dirigeante de l’associat...","[{""id"": 0, ""mentions"": [{""value"": ""pollution"",..."
51492,"Monsieur Elirone Trameur, membre de l’associat...","[{""id"": 0, ""mentions"": [{""value"": ""pollution"",..."


## Analysing raw data

### Check value types

In [4]:
labeled_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 800 entries, 181 to 124
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       800 non-null    object
 1   entities   800 non-null    object
 2   relations  800 non-null    object
dtypes: object(3)
memory usage: 25.0+ KB


In [5]:
test_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 1204 to 51742
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      400 non-null    object
 1   entities  400 non-null    object
dtypes: object(2)
memory usage: 9.4+ KB


In [6]:
# types of values in train data
print_value_types(labeled_raw_df)

text <class 'str'>
entities <class 'str'>
relations <class 'str'>


In [7]:
# types of values in test data
print_value_types(test_raw_df)

text <class 'str'>
entities <class 'str'>


### Check for duplicated rows

In [8]:
labeled_raw_df[labeled_raw_df.duplicated()]

,text,entities,relations
id,,,


In [9]:
test_raw_df[test_raw_df.duplicated()]

,text,entities
id,,


### Check for missing values

In [10]:
labeled_raw_df.isnull().sum()

text         0
entities     0
relations    0
dtype: int64

In [11]:
test_raw_df.isnull().sum()

text        0
entities    0
dtype: int64

In [12]:
labeled_raw_df.relations.sort_values()

id
41601    [[0, "DEATHS_NUMBER", 14], [3, "RESIDES_IN", 4...
2514     [[0, "END_DATE", 11], [0, "IS_LOCATED_IN", 4],...
1223     [[0, "END_DATE", 11], [10, "START_DATE", 11], ...
2329     [[0, "END_DATE", 11], [4, "RESIDES_IN", 1], [0...
3898     [[0, "END_DATE", 9], [5, "OPERATES_IN", 7], [1...
                               ...                        
4964     [[9, "IS_LOCATED_IN", 5], [8, "OPERATES_IN", 5...
41071    [[9, "IS_LOCATED_IN", 5], [9, "IS_LOCATED_IN",...
31669    [[9, "IS_LOCATED_IN", 8], [11, "OPERATES_IN", ...
41918    [[9, "IS_LOCATED_IN", 8], [13, "HAS_CONTROL_OV...
155      [[9, "OPERATES_IN", 1], [0, "IS_LOCATED_IN", 4...
Name: relations, Length: 800, dtype: object

## Data Cleaning/Wrangling

### Fix value typing

In [15]:
labeled_clean_df = clean_raw_dataset(labeled_raw_df)
test_clean_df = clean_raw_dataset(test_raw_df)

In [22]:
assert (labeled_raw_df.index == labeled_clean_df.index).all()
assert (test_raw_df.index == test_clean_df.index).all()

In [24]:
test_raw_df.head(1)

,text,entities
id,,
1204,“FEAR” est une organisation spécialisée dans l...,"[{""id"": 0, ""mentions"": [{""value"": ""crash"", ""st..."


In [23]:
test_clean_df.head(1)

,text,entities,relations
id,,,
1204,“FEAR” est une organisation spécialisée dans l...,"[{'id': 0, 'mentions': [{'value': 'crash', 'st...",None


### Check value types

In [18]:
# types of values in train data
print_value_types(labeled_clean_df)

text <class 'str'>
entities [  [<class 'str'>, <class 'str'>, <class 'str'>]  ]
relations [  <class 'list'>  ]


In [19]:
# types of values in test data
print_value_types(test_clean_df)

text <class 'str'>
entities [  [<class 'str'>, <class 'str'>, <class 'str'>]  ]
relations <class 'NoneType'>


### Save cleaned datasets

For manual analysis

In [25]:
save_data(labeled_clean_df, os.path.join(INTERIM_DIR, "train_cleaned.csv"))
save_data(test_clean_df, os.path.join(INTERIM_DIR, "test_cleaned.csv"))

## Analyze clean data

### All relation types

In [26]:
relation_classes = set(
    sum(
        labeled_clean_df.relations.apply(
            lambda row: list({r for (e1, r, e2) in row})
        ).values.tolist(),
        [],
    )
)
print(len(relation_classes))
relation_classes

37


{'CREATED',
 'DEATHS_NUMBER',
 'DIED_IN',
 'END_DATE',
 'GENDER_FEMALE',
 'GENDER_MALE',
 'HAS_CATEGORY',
 'HAS_COLOR',
 'HAS_CONSEQUENCE',
 'HAS_CONTROL_OVER',
 'HAS_FAMILY_RELATIONSHIP',
 'HAS_FOR_HEIGHT',
 'HAS_FOR_LENGTH',
 'HAS_FOR_WIDTH',
 'HAS_LATITUDE',
 'HAS_LONGITUDE',
 'HAS_QUANTITY',
 'INITIATED',
 'INJURED_NUMBER',
 'IS_AT_ODDS_WITH',
 'IS_BORN_IN',
 'IS_BORN_ON',
 'IS_COOPERATING_WITH',
 'IS_DEAD_ON',
 'IS_IN_CONTACT_WITH',
 'IS_LOCATED_IN',
 'IS_OF_NATIONALITY',
 'IS_OF_SIZE',
 'IS_PART_OF',
 'IS_REGISTERED_AS',
 'OPERATES_IN',
 'RESIDES_IN',
 'STARTED_IN',
 'START_DATE',
 'WAS_CREATED_IN',
 'WAS_DISSOLVED_IN',
 'WEIGHS'}

#### Summary stats on relation classes

TODO...

### All entity types

In [27]:
entity_classes = set(
    sum(
        labeled_clean_df.entities.apply(
            lambda row: list({e["type"] for e in row})
        ).values.tolist(),
        [],
    )
)
print(len(entity_classes))
entity_classes

55


{'ACCIDENT',
 'AGITATING_TROUBLE_MAKING',
 'BOMBING',
 'CATEGORY',
 'CBRN_EVENT',
 'CIVILIAN',
 'CIVIL_WAR_OUTBREAK',
 'COLOR',
 'COUP_D_ETAT',
 'CRIMINAL_ARREST',
 'DEMONSTRATION',
 'DRUG_OPERATION',
 'ECONOMICAL_CRISIS',
 'ELECTION',
 'EPIDEMIC',
 'FIRE',
 'FIRSTNAME',
 'GATHERING',
 'GROUP_OF_INDIVIDUALS',
 'HEIGHT',
 'HOOLIGANISM_TROUBLEMAKING',
 'ILLEGAL_CIVIL_DEMONSTRATION',
 'INTERGOVERNMENTAL_ORGANISATION',
 'LASTNAME',
 'LATITUDE',
 'LENGTH',
 'LONGITUDE',
 'MATERIAL_REFERENCE',
 'MATERIEL',
 'MILITARY',
 'MILITARY_ORGANISATION',
 'NATIONALITY',
 'NATURAL_CAUSES_DEATH',
 'NATURAL_EVENT',
 'NON_GOVERNMENTAL_ORGANISATION',
 'NON_MILITARY_GOVERNMENT_ORGANISATION',
 'PLACE',
 'POLITICAL_VIOLENCE',
 'POLLUTION',
 'QUANTITY_EXACT',
 'QUANTITY_FUZZY',
 'QUANTITY_MAX',
 'QUANTITY_MIN',
 'RIOT',
 'STRIKE',
 'SUICIDE',
 'TERRORIST_OR_CRIMINAL',
 'THEFT',
 'TIME_EXACT',
 'TIME_FUZZY',
 'TIME_MAX',
 'TIME_MIN',
 'TRAFFICKING',
 'WEIGHT',
 'WIDTH'}

#### Summary stats on entity classes

TODO...

## Split the labeled data into train-validation sets

In [31]:
# shuffling the dataset
labeled_clean_df = labeled_clean_df.sample(frac=1.0)
# split
VAL_SIZE = int(labeled_clean_df.shape[0] * VALIDATION_RATE)
val_clean_df = labeled_clean_df.head(VAL_SIZE)
TRAIN_SIZE = labeled_clean_df.shape[0] - VAL_SIZE
train_clean_df = labeled_clean_df.tail(TRAIN_SIZE)

assert len(train_clean_df.index.intersection(val_clean_df.index)) == 0
assert len(labeled_clean_df.index.difference(train_clean_df.index.union(val_clean_df.index))) == 0
labeled_clean_df.shape, train_clean_df.shape, val_clean_df.shape

((800, 3), (600, 3), (200, 3))

In [61]:
val_clean_df.head()

,text,entities,relations
id,,,
41802,Madame Destresse Cléa et deux membres de l’ass...,"[{'id': 0, 'mentions': [{'value': 'épidémie', ...","[[15, IS_LOCATED_IN, 1], [5, IS_LOCATED_IN, 1]..."
41069,Le meeting du parti au pouvoir sur le campus d...,"[{'id': 0, 'mentions': [{'value': 'meeting', '...","[[12, IS_AT_ODDS_WITH, 4], [13, IS_AT_ODDS_WIT..."
121,"Madame Lee Adilia, présidente de l’association...","[{'id': 0, 'mentions': [{'value': 'conférence'...","[[12, STARTED_IN, 6], [7, IS_LOCATED_IN, 6], [..."
1192,"Membres de l'association ""Mexico Limpio"", le M...","[{'id': 0, 'mentions': [{'value': 'crash', 'st...","[[0, STARTED_IN, 6], [7, IS_LOCATED_IN, 6], [1..."
41112,"Le 13 octobre 2020, un camion d’œuvres d’art v...","[{'id': 0, 'mentions': [{'value': 'volées', 's...","[[0, STARTED_IN, 12], [1, END_DATE, 18], [9, I..."


### Text length

#### Number of chars

In [32]:
pd.DataFrame(
    {
        set_name: df.text.apply(len).describe()
        for set_name, df in zip(
            ["train", "val", "test"],
            [train_clean_df, val_clean_df, test_clean_df],
        )
    }
)

,train,val,test
count,600.000000,200.000000,400.00000
mean,709.755000,698.555000,710.47000
std,122.931004,108.633481,110.84503
min,335.000000,394.000000,153.00000
25%,628.750000,634.000000,639.00000
50%,690.000000,680.500000,695.00000
75%,771.000000,744.250000,768.00000
max,1501.000000,1073.000000,1091.00000


#### Number of tokens (subwords from CamemBERT tokenizer)

In [34]:
pd.DataFrame(
    {
        set_name: df.text.apply(count_tokens).describe()
        for set_name, df in zip(
            ["train", "val", "test"],
            [train_clean_df, val_clean_df, test_clean_df],
        )
    }
)

,train,val,test
count,600.000000,200.000000,400.000000
mean,159.246667,156.235000,159.272500
std,27.330155,25.076161,24.978452
min,78.000000,83.000000,38.000000
25%,142.000000,139.750000,142.000000
50%,154.000000,152.500000,155.000000
75%,173.000000,168.000000,171.250000
max,322.000000,243.000000,256.000000


## Generate interim datasets

for each text, generate a csv file containing all the generated texts annotated as a multilabeled Text classification task

The csv file is named after the index of the text in the raw data


In [36]:
generated_data_dir_path = os.path.join(INTERIM_DIR, "multilabel_tagged_text_dataset")
assert not os.path.exists(
    generated_data_dir_path
), f"You must delete this folder first {generated_data_dir_path}!"

data_generator = TextToMultiLabelDataGenerator()

for split_name, clean_df in zip(
    ["test", "train", "val"],
    [test_clean_df, train_clean_df, val_clean_df],
):
    dest_dir_path = os.path.join(generated_data_dir_path, split_name)
    for multilabel_data in (
        pb := tqdm(
            data_generator.generate_row_multilabel_data(clean_df, only_w_relation=True if split_name!="test" else False),
            total=clean_df.shape[0],
            desc=f"{dest_dir_path} <- ",
        )
    ):
        text_index = multilabel_data.iloc[0][data_generator.text_index_col]
        dest_csv_file = os.path.join(dest_dir_path, f"{text_index}.csv")
        pb.set_description(f"{dest_csv_file} <-")
        save_data(multilabel_data, dest_csv_file, False)
        # break
    # break
# multilabel_data

data/defi-text-mine-2025/interim/multilabel_tagged_text_dataset/test/51742.csv <-: 100%|██████████| 400/400 [00:21<00:00, 19.02it/s]
data/defi-text-mine-2025/interim/multilabel_tagged_text_dataset/train/31620.csv <-: 100%|██████████| 600/600 [00:32<00:00, 18.52it/s]
data/defi-text-mine-2025/interim/multilabel_tagged_text_dataset/val/2382.csv <-: 100%|██████████| 200/200 [00:10<00:00, 18.94it/s] 


In [37]:
val_clean_df.loc[15]

text         La commune de Matéri est endeuillée suite à un...
entities     [{'id': 0, 'mentions': [{'value': 'incident', ...
relations    [[3, GENDER_MALE, 3], [5, IS_LOCATED_IN, 7], [...
Name: 15, dtype: object

## Check tagged text size using a  data loader to load data from csv files

to know wether the will fit at the input of the model (i.e. max of 512 tokens)

##### add special tokens to the tokenizer

In [38]:
# define special tokens to add to the tokenizer
task_special_tokens = ["<e1>", "</e1>", "<e2>", "</e2>"] + [
    f"<{entity_class}>" for entity_class in entity_classes
]
# task_special_tokens

In [39]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# add special tokens to the tokenizer
num_added_tokens = tokenizer.add_tokens(task_special_tokens, special_tokens=True)
num_added_tokens

/home/tagny/.pyenv/versions/3.11.8/envs/kgl11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


59

In [40]:
tokenizer

CamembertTokenizerFast(name_or_path='camembert/camembert-large', vocab_size=32005, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['<s>NOTUSED', '</s>NOTUSED', '<unk>NOTUSED']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>NOTUSED", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>NOTUSED", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<unk>NOTUSED", rstrip=False, lstrip=False, single_word=False

### initialize the data loader

In [41]:
interim_dataset = load_dataset(
    "csv",
    name="multilabel_tagged_text_dataset",
    data_dir="data/defi-text-mine-2025/interim/multilabel_tagged_text_dataset/",
    streaming=False,
)

Generating train split: 18690 examples [00:00, 29965.50 examples/s]
Generating validation split: 6372 examples [00:00, 30915.14 examples/s]
Generating test split: 174580 examples [00:01, 118483.39 examples/s]


In [42]:
interim_dataset

DatasetDict({
    train: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'relations'],
        num_rows: 18690
    })
    validation: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'relations'],
        num_rows: 6372
    })
    test: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'relations'],
        num_rows: 174580
    })
})

In [43]:
interim_dataset["train"][0]

{'text_index': 11,
 'e1': 0,
 'e2': 1,
 'text': "Le 2 juillet 2022, un <e1><ACCIDENT>accident</e1> s'est produit entre une moto et un bus sur la <e2><PLACE>RN 174</e2> à Saint-Lô. Ce dernier transportait 30 passagers issus du syndicat de lutte contre le racisme. Un panneau de signalisation a été <e1><ACCIDENT>percuté</e1> et 2 personnes sont mortes sur le coup. Le Président du Syndicat, Monsieur Smith Aarone, était parmi les blessés. Il a immédiatement été transporté en hélicoptère à l'hôpital. Ayant conduit sans permis et en état d’ivresse, le conducteur de la moto a été retrouvé mort au bas d'un talus. Les journalistes sont ensuite arrivés sur le <e2><PLACE>lieu</e2> avec leurs caméras, pour interroger les policiers sur les causes de l'<e1><ACCIDENT>accident</e1>.",
 'relations': "{'IS_LOCATED_IN', 'STARTED_IN'}"}

In [44]:
interim_dataset["train"][1]

{'text_index': 11,
 'e1': 0,
 'e2': 2,
 'text': "Le 2 juillet 2022, un <e1><ACCIDENT>accident</e1> s'est produit entre une moto et un bus sur la RN 174 à <e2><PLACE>Saint-Lô</e2>. Ce dernier transportait 30 passagers issus du syndicat de lutte contre le racisme. Un panneau de signalisation a été <e1><ACCIDENT>percuté</e1> et 2 personnes sont mortes sur le coup. Le Président du Syndicat, Monsieur Smith Aarone, était parmi les blessés. Il a immédiatement été transporté en hélicoptère à l'hôpital. Ayant conduit sans permis et en état d’ivresse, le conducteur de la moto a été retrouvé mort au bas d'un talus. Les journalistes sont ensuite arrivés sur le lieu avec leurs caméras, pour interroger les policiers sur les causes de l'<e1><ACCIDENT>accident</e1>.",
 'relations': "{'IS_LOCATED_IN'}"}

In [45]:
interim_dataset["validation"][7]

{'text_index': 1108,
 'e1': 12,
 'e2': 1,
 'text': "Le 22 mai 2021, un <e2><CIVILIAN>ouvrier</e2> a été blessé suite à une chute sur un échafaudage de cinq mètres de hauteur alors qu’<e2><CIVILIAN>il</e2> travaillait sur un chantier de construction à Villeurbanne. Heureusement, le contrat d’assurance signé avec l'agent général d'assurance Monsieur Dos Santos Dominique stipulait que les frais de soins médicaux étaient totalement pris en charge. Une fois vérification par téléphone avec ce dernier, les <e1><NON_MILITARY_GOVERNMENT_ORGANISATION>secours</e1> ont été appelés et l’ambulance est arrivée dans la foulée pour évacuer le <e2><CIVILIAN>blessé</e2>. Suite à cet accident, les mesures de sécurité sur le chantier ont été améliorées, de nouveaux garde-corps ont été mis en place au niveau des échafaudages, et les ouvriers ont été obligés de porter des casques et des harnais de sécurité pour les travaux en hauteur.",
 'relations': "{'IS_IN_CONTACT_WITH'}"}

In [46]:
interim_dataset["test"][7]

{'text_index': 1102,
 'e1': 0,
 'e2': 3,
 'text': "Le 5 février 2022 à Marseille, un automobiliste âgé de 35 ans est <e1><ACCIDENT>mort</e1> dans sa <e2><MATERIEL>voiture</e2> après avoir <e1><ACCIDENT>percuté</e1> un bâtiment. Afin de minimiser les dégâts, des engins et des équipements spécialisés ont été utilisés pour retirer la <e2><MATERIEL>voiture</e2> du bâtiment. Après constat, l'inspecteur Charles Laurent de la police scientifique a détecté la présence d'alcool et de drogue dans le sang du conducteur. Ce dernier correspondait au signalement dans les avis de recherche de la police : homme mince, roux et aux cheveux courts. Il y a quelques jours, les habitants avaient remarqué un homme ressemblant au suspect, vêtu d'une chemise bleue et d'un pantalon noir, muni d'une arme à feu.",
 'relations': None}

### Count tokens

In [47]:
def count_token_in_dataset_element(example: Dict[str, Any]) -> Dict[str, int]:
    return {"n_tokens": count_tokens(example["text"])}


interim_dataset = interim_dataset.map(count_token_in_dataset_element)
interim_dataset

Map: 100%|██████████| 174580/174580 [00:45<00:00, 3808.09 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'relations', 'n_tokens'],
        num_rows: 18690
    })
    validation: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'relations', 'n_tokens'],
        num_rows: 6372
    })
    test: Dataset({
        features: ['text_index', 'e1', 'e2', 'text', 'relations', 'n_tokens'],
        num_rows: 174580
    })
})

In [49]:
interim_dataset["test"][0]

{'text_index': 1102,
 'e1': 1,
 'e2': 0,
 'text': "Le 5 février 2022 à <e1><PLACE>Marseille</e1>, un automobiliste âgé de 35 ans est <e2><ACCIDENT>mort</e2> dans sa voiture après avoir <e2><ACCIDENT>percuté</e2> un bâtiment. Afin de minimiser les dégâts, des engins et des équipements spécialisés ont été utilisés pour retirer la voiture du bâtiment. Après constat, l'inspecteur Charles Laurent de la police scientifique a détecté la présence d'alcool et de drogue dans le sang du conducteur. Ce dernier correspondait au signalement dans les avis de recherche de la police : homme mince, roux et aux cheveux courts. Il y a quelques jours, les habitants avaient remarqué un homme ressemblant au suspect, vêtu d'une chemise bleue et d'un pantalon noir, muni d'une arme à feu.",
 'relations': None,
 'n_tokens': 151}

In [50]:
split2ntokens_df = pd.DataFrame(
    {
        split_name: pd.Series(
            [e["n_tokens"] for e in tqdm(interim_dataset[split_name], split_name)],
            name=f"{split_name}_text_n_tokens",
        ).describe()
        for split_name in interim_dataset.keys()
    }
)
split2ntokens_df

test: 100%|██████████| 174580/174580 [00:04<00:00, 42912.00it/s]


,train,validation,test
count,18690.000000,6372.000000,174580.000000
mean,175.342429,170.142341,172.616107
std,30.798631,27.199698,26.667400
min,85.000000,86.000000,44.000000
25%,155.000000,152.000000,153.000000
50%,169.000000,167.000000,168.000000
75%,190.000000,182.000000,188.000000
max,369.000000,282.000000,297.000000


### Getting ready for submission

In [59]:
text_indexes = {sample["text_index"] for sample in interim_dataset["test"]}
print(str(text_indexes))

{51211, 51212, 13, 51214, 51215, 18, 19, 51222, 51223, 51227, 51229, 51239, 41000, 41001, 51241, 51242, 51243, 41006, 41010, 51251, 51257, 41025, 41026, 41034, 41038, 51281, 51290, 41051, 51297, 41058, 41068, 51308, 51311, 114, 41074, 51315, 51317, 119, 41080, 122, 41083, 41084, 51322, 41087, 41088, 51327, 132, 41092, 41094, 51333, 138, 41099, 41102, 51344, 51345, 147, 41108, 41110, 151, 51350, 41113, 154, 41114, 41115, 51354, 41118, 160, 51363, 41124, 41125, 170, 51371, 41134, 41135, 51376, 180, 41140, 41141, 51384, 51386, 41147, 41148, 51390, 41151, 51392, 193, 194, 195, 41153, 41154, 51394, 51395, 51397, 51400, 41162, 41163, 41164, 51401, 51405, 51409, 41171, 41173, 41176, 51416, 41178, 51420, 41184, 41185, 51426, 41188, 51431, 41195, 41198, 51441, 51443, 51445, 51446, 51448, 51449, 51452, 51453, 51455, 51458, 2307, 2308, 51459, 2313, 2319, 2324, 51476, 51480, 2336, 51491, 51492, 51495, 2345, 2346, 51498, 2350, 2351, 2353, 2360, 2362, 2372, 2373, 2375, 51531, 2381, 2394, 2397, 2400,

In [60]:
text_index = 1204
for example in interim_dataset["test"]:
    if example['text_index'] != text_index:
        continue
    print(example)
    break

{'text_index': 1204, 'e1': 1, 'e2': 0, 'text': "“FEAR” est une organisation spécialisée dans le transport des tablettes et des smartphones basée à Buenos Aires, d'une latitude de -34.6083. Il s'agit de la plus grande métropole cosmopolite de l'Argentine. Il a été rapporté que le dernier avion commercial de “FEAR” a fait un <e2><ACCIDENT>crash</e2> aérien en novembre dernier. Cela est dû à une perte de contrôle de l'appareil. Au bord du vol se trouvaient les pilotes M. <e1><CIVILIAN>Clavier Joshua</e1> et M. Joseph Charles. Tous deux ont perdu la vie dans ce terrible <e2><ACCIDENT>accident</e2>. Le serveur informatique et le signal émis par la satellite ont permis aux secours de localiser l'appareil.", 'relations': None, 'n_tokens': 146}


In [62]:
text_index = 1102
for example in interim_dataset["test"]:
    if example['text_index'] != text_index:
        continue
    print(example)
    break

{'text_index': 1102, 'e1': 1, 'e2': 0, 'text': "Le 5 février 2022 à <e1><PLACE>Marseille</e1>, un automobiliste âgé de 35 ans est <e2><ACCIDENT>mort</e2> dans sa voiture après avoir <e2><ACCIDENT>percuté</e2> un bâtiment. Afin de minimiser les dégâts, des engins et des équipements spécialisés ont été utilisés pour retirer la voiture du bâtiment. Après constat, l'inspecteur Charles Laurent de la police scientifique a détecté la présence d'alcool et de drogue dans le sang du conducteur. Ce dernier correspondait au signalement dans les avis de recherche de la police : homme mince, roux et aux cheveux courts. Il y a quelques jours, les habitants avaient remarqué un homme ressemblant au suspect, vêtu d'une chemise bleue et d'un pantalon noir, muni d'une arme à feu.", 'relations': None, 'n_tokens': 151}
